In [1]:
from CybORG import CybORG
from CybORG.Simulator.Scenarios import EnterpriseScenarioGenerator
from CybORG.Agents.Wrappers import EnterpriseMAE
from CybORG.Agents import SleepAgent, EnterpriseGreenAgent, FiniteStateRedAgent
from Wrappers import EnterpriseMAEMaskWrapper, MaskWrapper
from CustomRLLib import TorchActionMaskModel, CustomModel

In [2]:
from ray.tune import register_env
from ray.rllib.algorithms.ppo import PPOConfig, PPO
from ray.rllib.algorithms.dqn import DQNConfig
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.models import ModelCatalog

In [3]:
def env_creator_CC4(env_config: dict):
    sg = EnterpriseScenarioGenerator(
        blue_agent_class=SleepAgent,
        green_agent_class=EnterpriseGreenAgent,
        red_agent_class=FiniteStateRedAgent,
        steps=500
    )
    cyborg = CybORG(scenario_generator=sg)
    # env = EnterpriseMAE(env=cyborg)
    # env = EnterpriseMAEMaskWrapper(env=cyborg)
    env = MaskWrapper(env=cyborg)

    return env

In [4]:
register_env(name='CC4', env_creator=lambda config: env_creator_CC4(config))
env = env_creator_CC4({})

In [5]:
NUM_AGENTS = 5
POLICY_MAP = {f"blue_agent_{i}": f"Agent{i}" for i in range(NUM_AGENTS)}

def policy_mapper(agent_id, episode, worker, **kwargs):
# def policy_mapper(agent_id, episode, **kwargs):
    return POLICY_MAP[agent_id]

In [6]:
ModelCatalog.register_custom_model("custom_pytorch_model", CustomModel)
ModelCatalog.register_custom_model("torch_action_mask_model", TorchActionMaskModel)
# ModelCatalog.register_custom_model("action_mask_model", ActionMaskModel)


# ray.init()
# algo = ppo.PPO(env="CartPole-v1", config={
#     "model": {
#         "custom_model": "my_tf_model",
#         # Extra kwargs to be passed to your model's c'tor.
#         "custom_model_config": {},
#     },
# })

In [7]:
algo_config = (
    PPOConfig()
    .environment(env="CC4")
    .debugging(logger_config={'logdir':'./logs/PPO_Example', 'type': 'ray.tune.logger.TBXLogger'})
    .multi_agent(policies={
        ray_agent: PolicySpec(
            policy_class=None,
            observation_space=env.observation_space(cyborg_agent),
            action_space=env.action_space(cyborg_agent),
            config={'gamma': 0.85}
        ) for cyborg_agent, ray_agent in POLICY_MAP.items()
    },
    policy_mapping_fn=policy_mapper
    )
    .training(
        model={'custom_model': "torch_action_mask_model"}
    )
)

In [8]:
print(algo_config.policies['Agent0'].policy_class)

None


In [9]:
import time
print(time.strftime("%Y-%m-%d_%H:%M:%S"))

2024-04-16_17:16:53


In [10]:
algo = algo_config.build()

for i in range(2):
    train_info=algo.train()

algo.save(f'./Submissions/results_{time.strftime("%Y-%m-%d_%H:%M:%S")}/staging/')

/Applications/anaconda3/envs/cage-4/lib/python3.10/site-packages/ray/rllib/utils/from_config.py:197: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `TBXLogger interface is deprecated in favor of the `ray.tune.tensorboardx.TBXLoggerCallback` interface and will be removed in Ray 2.7.
  object_ = constructor(*ctor_args, **ctor_kwargs)
2024-04-16 17:16:56,971	INFO worker.py:1724 -- Started a local Ray instance.
(RolloutWorker pid=46111) 2024-04-16 17:16:59,685	WARNING env.py:298 -- Your MultiAgentEnv <EnterpriseMAEMaskWrapper instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2024-04-16 17:17:00,677	WARNING util.py:62 -- Install gputil for GPU system monitoring.
(RolloutWorker pid=46111) /Applications/

!!!! space_to_dict(self.observation_space):  {'space': {'space': 'box', 'low': 'eJyb7BfqGxDJyFDGUK2eklqcXKRupaBuk2airqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RpGZuY6mjoKtQpkAy4Ghob9o3gUj+JRTAgDAJnVaEo=', 'high': 'eJyb7BfqGxDJyFDGUK2eklqcXKRupaBuk2airqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RpGZuY6mjoKtQpkAy4Ghgb7UTyKR/EoJoQBlgviuw==', 'shape': (267,), 'dtype': '<f4'}, 'original_space': {'space': {'space': 'dict', 'spaces': {'action_mask': {'space': 'multi-binary', 'n': 82}, 'observations': {'space': 'multi-discrete', 'nvec': 'eJyb7BfqGxDJyFDGUK2eklqcXKRupaBuk2mhrqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RqWRjqaOgq1CuQDLmYGCGAapUfpYUgDAGpCHDI=', 'dtype': '<i8'}}}}}
!!!! space_to_dict(self.observation_space):  {'space': {'space': 'box', 'low': 'eJyb7BfqGxDJyFDGUK2eklqcXKRupaBuk2airqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RpmZsY6mjoKtQpkAy4Ghob9o3gUj+JRPIpH8SgexaN4FFMDAwDaXlXc', 'high': 'eJyb7BfqGxDJyFDGUK2eklqcXKRupaBuk2airqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RpmZsY6mjoKtQpkAy4Ghgb7UTyKR/E

TrainingResult(checkpoint=Checkpoint(filesystem=local, path=./Submissions/results_2024-04-16_17:19:10/staging/), metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'Agent3': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.8597488654466967, 'cur_kl_coeff': 0.29999999999999993, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 9.936818678180376, 'policy_loss': -0.02997081426656223, 'vf_loss': 9.962491466601689, 'vf_explained_var': -1.2999152143796285e-05, 'kl': 0.014326764248146735, 'entropy': 3.8729896634817123, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 125.0, 'num_grad_updates_lifetime': 1440.5, 'diff_num_grad_updates_vs_sampler_policy': 479.5}, 'Agent4': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.8690984778106212, 'cur_kl_coeff': 0.29999999999999993, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 9.911309420069058, 'policy_loss': -0.058836115999535345, 'vf_loss': 9.963830057779948, 'vf_expla

In [11]:
policy = algo.get_policy(policy_id='Agent0')
# policy.compute_actions()
print(env.observation_space('blue_agent_0'))
print(len(env.observation_space('blue_agent_0').sample()))
print(env.action_space('blue_agent_0'))
print(policy.observation_space)

Dict('action_mask': MultiBinary(82), 'observations': MultiDiscrete([3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]))
2
Discrete(82)
Box(-1.0, 1.0, (267,), float32)


In [12]:
print(env.observation_space('blue_agent_0').sample())

OrderedDict([('action_mask', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0], dtype=int8)), ('observations', array([1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0]))])


In [13]:
print(env.observation_space('blue_agent_4'))

Dict('action_mask': MultiBinary(242), 'observations': MultiDiscrete([3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]))


In [14]:
from ray.rllib.policy.policy import PolicySpec, Policy
my_restored_policy = Policy.from_checkpoint("./Submissions/mask_results_2024-04-16_02:01:29/staging/policies/Agent0")